# Chapter 7. 텍스트 문서의 범주화 - (4) IMDB 데이터에 전이학습 적용하기

- 작업에 필요한 <b>레이블이 있는 데이터</b>가 적은데 비슷한 <b>다른 도메인의 훈련 데이터</b>는 많을 경우, 전이학습이 유용하다.
- 이제 전이학습을 이용하여 불충분한 데이터셋으로도 CNN 모델을 학습시켜보자.
    - IMDB 영화리뷰 데이터셋에서 일부(5%)만 추출하여 불충분한 데이터셋으로 사용한다
- IMDB 영화 리뷰 데이터를 다운로드 받아 data 디렉토리에 압축 해제한다
    - 다운로드 : http://ai.stanford.edu/~amaas/data/sentiment/
    - 저장경로 : data/aclImdb

In [1]:
import os
import config
from dataloader.loader import Loader
from preprocessing.utils import Preprocess, remove_empty_docs
from dataloader.embeddings import GloVe
from model.cnn_document_model import DocumentModel, TrainingParameters
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

Using TensorFlow backend.


## 학습 파라미터 설정

In [2]:
# 학습된 모델을 저장할 디렉토리 생성
if not os.path.exists(os.path.join(config.MODEL_DIR, 'imdb')):
    os.makedirs(os.path.join(config.MODEL_DIR, 'imdb'))

# 학습 파라미터 설정
train_params = TrainingParameters('imdb_transfer_tanh_activation', 
                                  model_file_path = config.MODEL_DIR+ '/imdb/transfer_model_10.hdf5',
                                  model_hyper_parameters = config.MODEL_DIR+ '/imdb/transfer_model_10.json',
                                  model_train_parameters = config.MODEL_DIR+ '/imdb/transfer_model_10_meta.json',
                                  num_epochs=30,
                                  batch_size=128)

## IMDB 데이터셋 로드

In [3]:
# 다운받은 IMDB 데이터 로드: 학습셋은 5%만 취한다 (전체는 2만5천개)
train_df = Loader.load_imdb_data(directory = 'train')
train_df = train_df.sample(frac=0.05, random_state = train_params.seed)
print(f'train_df.shape : {train_df.shape}')

test_df = Loader.load_imdb_data(directory = 'test')
print(f'test_df.shape : {test_df.shape}')

# 텍스트 데이터, 레이블 추출
corpus = train_df['review'].tolist()
target = train_df['sentiment'].tolist()
corpus, target = remove_empty_docs(corpus, target)
print(f'corpus size : {len(corpus)}')
print(f'target size : {len(target)}')

train_df.shape : (1250, 2)
test_df.shape : (25000, 2)
corpus size : 1250
target size : 1250


## 인덱스 시퀀스 생성

In [4]:
# 학습셋을 인덱스 시퀀스로 변환
preprocessor = Preprocess(corpus=corpus)
corpus_to_seq = preprocessor.fit()

Found 4917 unique tokens.
All documents processed.cessed.

In [5]:
print(f'corpus_to_seq size : {len(corpus_to_seq)}')
print(f'corpus_to_seq[0] size : {len(corpus_to_seq[0])}')

corpus_to_seq size : 1250
corpus_to_seq[0] size : 300


In [6]:
# 테스트셋을 인덱스 시퀀스로 변환
test_corpus = test_df['review'].tolist()
test_target = test_df['sentiment'].tolist()
test_corpus, test_target = remove_empty_docs(test_corpus, test_target)
test_corpus_to_seq = preprocessor.transform(test_corpus)

All documents processed.ocessed.

In [7]:
print(f'test_corpus_to_seq size : {len(test_corpus_to_seq)}')
print(f'test_corpus_to_seq[0] size : {len(test_corpus_to_seq[0])}')

test_corpus_to_seq size : 25000
test_corpus_to_seq[0] size : 300


In [8]:
# 학습셋, 테스트셋 준비
x_train = np.array(corpus_to_seq)
x_test = np.array(test_corpus_to_seq)
y_train = np.array(target)
y_test = np.array(test_target)

print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_test.shape : {x_test.shape}')
print(f'y_test.shape : {y_test.shape}')

x_train.shape : (1250, 300)
y_train.shape : (1250,)
x_test.shape : (25000, 300)
y_test.shape : (25000,)


## GloVe 임베딩 초기화

In [9]:
# GloVe 임베딩 초기화 - glove.6B.50d.txt pretrained 벡터 사용
glove = GloVe(50)
initial_embeddings = glove.get_embedding(preprocessor.word_index)
print(f'initial_embeddings.shape : {initial_embeddings.shape}')

Reading 50 dim GloVe vectors
Found 400000 word vectors.
words not found in embeddings: 10
initial_embeddings.shape : (4919, 50)


##  훈련된 모델 로드

- HandsOn03에서 아마존 리뷰 데이터로 학습한 CNN 모델을 로드한다.
- DocumentModel 클래스의 load_model로 모델을 로드하고, load_model_weights로 학습된 가중치를 가져온다. 
- 그 후, GloVe.update_embeddings 함수로 GloVe 초기화 임베딩을 업데이트한다

In [10]:
# 모델 하이퍼파라미터 로드
model_json_path = os.path.join(config.MODEL_DIR, 'amazonreviews/model_06.json')
amazon_review_model = DocumentModel.load_model(model_json_path)

# 모델 가중치 로드
model_hdf5_path = os.path.join(config.MODEL_DIR, 'amazonreviews/model_06.hdf5')
amazon_review_model.load_model_weights(model_hdf5_path)


Vocab Size = 43197  and the index of vocabulary words passed has 43195 words
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
# 모델 임베딩 레이어 추출
learned_embeddings = amazon_review_model.get_classification_model().get_layer('imdb_embedding').get_weights()[0]
print(f'learned_embeddings size : {len(learned_embeddings)}')

# 기존 GloVe 모델을 학습된 임베딩 행렬로 업데이트한다
glove.update_embeddings(preprocessor.word_index, 
                        np.array(learned_embeddings), 
                        amazon_review_model.word_index)

# 업데이트된 임베딩을 얻는다
initial_embeddings = glove.get_embedding(preprocessor.word_index)


learned_embeddings size : 43197
4839 words are updated out of 4917


##  IMDB 전이학습 모델 생성

In [12]:
# 분류 모델 생성 : IMDB 리뷰 데이터를 입력받아 이진분류를 수행하는 모델 생성
imdb_model = DocumentModel(vocab_size=preprocessor.get_vocab_size(),
                           word_index = preprocessor.word_index,
                           num_sentences=Preprocess.NUM_SENTENCES,     
                           embedding_weights=initial_embeddings,
                           embedding_regularizer_l2 = 0.0,
                           conv_activation = 'tanh',
                           train_embedding = False,  # 임베딩 레이어의 가중치 학습 안 함
                           learn_word_conv = False,  # 단어 수준 conv 레이어의 가중치 학습 안 함
                           learn_sent_conv = False,  # 문장 수준 conv 레이어의 가중치 학습 안 함
                           hidden_dims=64,                                        
                           input_dropout=0.1, 
                           hidden_layer_kernel_regularizer=0.01,
                           final_layer_kernel_regularizer=0.01)

# 가중치 업데이트 : 생성한 imdb_model 모델에서 다음의 각 레이어들의 가중치를 위에서 로드한 가중치로 갱신한다
for l_name in ['word_conv','sentence_conv','hidden_0', 'final']:
    new_weights = amazon_review_model.get_classification_model().get_layer(l_name).get_weights()
    imdb_model.get_classification_model().get_layer(l_name).set_weights(weights=new_weights)

Vocab Size = 4919  and the index of vocabulary words passed has 4917 words


## 모델 학습 및 평가

In [13]:
# 모델 컴파일              
imdb_model.get_classification_model().compile(loss="binary_crossentropy", 
                                              optimizer='rmsprop',
                                              metrics=["accuracy"])

# callback (1) - 체크포인트
checkpointer = ModelCheckpoint(filepath=train_params.model_file_path,
                                verbose=1,
                                save_best_only=True,
                                save_weights_only=True)

# callback (2) - 조기종료
early_stop = EarlyStopping(patience=2)

# 학습 시작
imdb_model.get_classification_model().fit(x_train, 
                                          y_train, 
                                          batch_size=train_params.batch_size,
                                          epochs=train_params.num_epochs,
                                          verbose=2,
                                          validation_split=0.01,
                                          callbacks=[checkpointer])

# 모델 저장
imdb_model._save_model(train_params.model_hyper_parameters)
train_params.save()

Instructions for updating:
Use tf.cast instead.
Train on 1237 samples, validate on 13 samples
Epoch 1/30
 - 1s - loss: 1.5930 - acc: 0.8424 - val_loss: 1.5610 - val_acc: 0.8462

Epoch 00001: val_loss improved from inf to 1.56104, saving model to ./checkpoint/imdb/transfer_model_10.hdf5
Epoch 2/30
 - 0s - loss: 1.4717 - acc: 0.8383 - val_loss: 1.4601 - val_acc: 0.8462

Epoch 00002: val_loss improved from 1.56104 to 1.46013, saving model to ./checkpoint/imdb/transfer_model_10.hdf5
Epoch 3/30
 - 0s - loss: 1.3991 - acc: 0.8448 - val_loss: 1.3791 - val_acc: 0.8462

Epoch 00003: val_loss improved from 1.46013 to 1.37906, saving model to ./checkpoint/imdb/transfer_model_10.hdf5
Epoch 4/30
 - 0s - loss: 1.3127 - acc: 0.8472 - val_loss: 1.3162 - val_acc: 0.8462

Epoch 00004: val_loss improved from 1.37906 to 1.31624, saving model to ./checkpoint/imdb/transfer_model_10.hdf5
Epoch 5/30
 - 0s - loss: 1.2514 - acc: 0.8529 - val_loss: 1.2432 - val_acc: 0.9231

Epoch 00005: val_loss improved from 1.

In [14]:
# 모델 평가
imdb_model.get_classification_model().evaluate(x_test, 
                                               y_test, 
                                               batch_size=train_params.batch_size*10,
                                               verbose=2)

[0.5280678803443909, 0.8523999965667725]

## SVM과의 성능 비교

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [16]:
# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True, 
                     min_df=0.00005, 
                     max_df=1.0, 
                     ngram_range=(1, 1), 
                     stop_words='english', 
                     sublinear_tf=True)

tv_features = tv.fit_transform(train_df['review'].tolist())
print(f'feature names size : {len(tv.get_feature_names())}')

tv_train_features = tv.transform(corpus)
tv_test_features = tv.transform(test_corpus)

# SVC 모델 학습 & 평가
clf = SVC(C=1,kernel='linear', random_state=1, gamma=0.01)
svm = clf.fit(tv_train_features, target)
preds_test = svm.predict(tv_test_features)


feature names size : 20340


In [17]:
print(accuracy_score(y_test, preds_test))

0.83128
